In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.set_printoptions(linewidth=1000, edgeitems=30)
pd.set_option('display.max_columns', 60)
pd.set_option('display.width', 2000)

In [ ]:
# Some StackOverflow magic to enable importing of local module
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from multiLevelCoSurrogates.local import base_dir

In [ ]:
records = ['ei0.0_records.csv', 'ei0.0_diff_records.csv',
           'ei0.5_records.csv', 'ei0.5_diff_records.csv', 
           'ei1.0_records.csv', 'ei1.0_diff_records.csv', 
           'ucb0.5_records.csv', 'ucb0.5_diff_records.csv', 
           'ucb1.0_records.csv', 'ucb1.0_diff_records.csv', 
           'ucb1.5_records.csv', 'ucb1.5_diff_records.csv', 
           'ucb2.0_records.csv', 'ucb2.0_diff_records.csv', 
           'ucb3.0_records.csv', 'ucb3.0_diff_records.csv', 
           'ucb4.0_records.csv', 'ucb4.0_diff_records.csv', 
           'ucb5.0_records.csv', 'ucb5.0_diff_records.csv', ]

print(os.listdir(base_dir))

df = pd.read_csv(base_dir+records[0], index_col='index')
df.head()

In [ ]:
fig, axes = plt.subplots(3,2, figsize=(9,9))

grouped = df.groupby(by=['which_model', 'fidelity'])

for ax, group in zip(axes.flatten(), grouped):
    name, sub_df = group
    sub_df = sub_df.groupby(by='iteration').mean()
    sub_df.plot(y=['mse_high', 'mse_low', 'mse_hier'], ax=ax)
    ax.set_title('acq on {}, {} fidelity updated'.format(*name))

plt.tight_layout()
plt.show()

In [ ]:
def plot_normalized(df):

    fig, axes = plt.subplots(3,2, figsize=(9,9))

    grouped = df.groupby(by=['which_model', 'fidelity'])
    mse_names = ['mse_high', 'mse_low', 'mse_hier']

    for ax, group in zip(axes.flatten(), grouped):
        name, sub_df = group

        # Separate out the MSE values after initial training to use as reference
        num_iters = len(sub_df['iteration'].unique())
        first_rows = sub_df.loc[0::num_iters,mse_names].values

        # Get all values and reshape without copying s.t. the iterations are a separate dimension
        values = sub_df.loc[:,mse_names].values
        old_shape = values.shape
        new_shape = old_shape[0]//num_iters, num_iters, len(mse_names)
        values.shape = new_shape

        # Now we can use numpy's internal broadcasting for efficient division, and return the old shape afterwards
        new_values = values / first_rows[:,None,:]
        new_values.shape = old_shape

        # Finally the values are returned to the dataframe and plotted
        sub_df.loc[:,mse_names] = new_values
        sub_df = sub_df.groupby(by='iteration').mean()
        sub_df.plot(y=mse_names, ax=ax)
        ax.set_title('acq on {}, {} fidelity updated'.format(*name))

    plt.tight_layout()
    plt.show()

In [ ]:
print(records[0])
plot_normalized(pd.read_csv(base_dir+records[0], index_col='index'))

In [ ]:
print(records[1])
plot_normalized(pd.read_csv(base_dir+records[1], index_col='index'))

In [ ]:
print(records[2])
plot_normalized(pd.read_csv(base_dir+records[2], index_col='index'))

In [ ]:
print(records[3])
plot_normalized(pd.read_csv(base_dir+records[3], index_col='index'))

In [ ]:
print(records[4])
plot_normalized(pd.read_csv(base_dir+records[4], index_col='index'))

In [ ]:
print(records[5])
plot_normalized(pd.read_csv(base_dir+records[5], index_col='index'))

In [ ]:
print(records[6])
plot_normalized(pd.read_csv(base_dir+records[6], index_col='index'))

In [ ]:
print(records[7])
plot_normalized(pd.read_csv(base_dir+records[7], index_col='index'))

In [ ]:
print(records[8])
plot_normalized(pd.read_csv(base_dir+records[8], index_col='index'))

In [ ]:
print(records[9])
plot_normalized(pd.read_csv(base_dir+records[9], index_col='index'))

In [ ]:
print(records[10])
plot_normalized(pd.read_csv(base_dir+records[10], index_col='index'))

In [ ]:
print(records[11])
plot_normalized(pd.read_csv(base_dir+records[11], index_col='index'))

In [ ]:
new_base_dir = '/media/rijnsjvan/Data/HRI_EGN_Presentation_data/bifidbo/'
new_records = ['ei_records.csv', 'ucb_records.csv']

In [ ]:
print(new_records[0])
plot_normalized(pd.read_csv(new_base_dir+new_records[0], index_col='index'))

In [ ]:
print(new_records[1])
plot_normalized(pd.read_csv(new_base_dir+new_records[1], index_col='index'))